In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, regularizers
import os

### Preparing data

In [3]:
df_train = pd.read_csv('train.csv')
X_train = np.array(df_train)[:, 1:]
assert X_train.shape == (42000, 28*28)
X_train = X_train.reshape((42000, 28, 28, 1))
y_train = np.array(df_train['label'])
y_train_cat = keras.utils.to_categorical(y_train, num_classes=10)

df_test = pd.read_csv('test.csv')
X_test =  np.array(df_test)
assert X_test.shape == (28000, 28*28)
X_test = X_test.reshape((28000, 28, 28, 1))

### Training

In [4]:
model = keras.Sequential([
layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, 
              activation = "relu", input_shape = (28,28,1), kernel_regularizer=regularizers.l2(0.0005)),
layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(pool_size = 2, strides = 2),
layers.Dropout(0.25),
layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = "relu", 
              kernel_regularizer=regularizers.l2(0.0005)),
layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.MaxPooling2D(pool_size = 2, strides = 2),
layers.Dropout(0.25),
layers.Flatten(),
layers.Dense(units = 256, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.Dense(units = 128, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.Dense(units = 84, use_bias=False),
layers.BatchNormalization(),
layers.Activation("relu"),
layers.Dropout(0.25),
layers.Dense(units = 10, activation = "softmax")
])
model.compile(optimizer="adam",
              loss=keras.losses.categorical_crossentropy,
              metrics=["accuracy"])

/home/fedimser/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.fit(X_train, y_train_cat, epochs=1, batch_size=1000, validation_split=0.1)      

38/38 ━━━━━━━━━━━━━━━━━━━━ 35s 861ms/step - accuracy: 0.5568 - loss: 1.4122 - val_accuracy: 0.5910 - val_loss: 2.0804


In [6]:
CHECKPOINT_PATH = "model3.weights.h5"
if os.path.exists(CHECKPOINT_PATH):
  model.load_weights(CHECKPOINT_PATH)
  print("Loaded weights.")

Loaded weights.


In [7]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                                 save_weights_only=True,
                                                 verbose=1)
_ = model.fit(X_train, y_train_cat, epochs=30, batch_size=100, validation_split=0.1, callbacks=[cp_callback])  

Epoch 1/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.9342 - loss: 0.2533
Epoch 1: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 32s 85ms/step - accuracy: 0.9343 - loss: 0.2531 - val_accuracy: 0.9817 - val_loss: 0.0779
Epoch 2/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.9765 - loss: 0.0926
Epoch 2: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 32s 85ms/step - accuracy: 0.9765 - loss: 0.0926 - val_accuracy: 0.9876 - val_loss: 0.0618
Epoch 3/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.9826 - loss: 0.0747
Epoch 3: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 33s 87ms/step - accuracy: 0.9826 - loss: 0.0747 - val_accuracy: 0.9838 - val_loss: 0.0604
Epoch 4/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.9848 - loss: 0.0658
Epoch 4: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 45s 119ms/step - accuracy: 0.9848 - loss: 0.0658 - val_accuracy: 0.9798 - val_loss: 0.0723
Ep

Epoch 27/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.9947 - loss: 0.0242
Epoch 27: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 32s 84ms/step - accuracy: 0.9947 - loss: 0.0242 - val_accuracy: 0.9933 - val_loss: 0.0286
Epoch 28/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.9952 - loss: 0.0234
Epoch 28: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 33s 86ms/step - accuracy: 0.9952 - loss: 0.0235 - val_accuracy: 0.9936 - val_loss: 0.0285
Epoch 29/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.9957 - loss: 0.0217
Epoch 29: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 33s 87ms/step - accuracy: 0.9957 - loss: 0.0217 - val_accuracy: 0.9931 - val_loss: 0.0339
Epoch 30/30
378/378 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.9943 - loss: 0.0260
Epoch 30: saving model to model3.weights.h5
378/378 ━━━━━━━━━━━━━━━━━━━━ 32s 85ms/step - accuracy: 0.9943 - loss: 0.0260 - val_accuracy: 0.9931 - val_loss: 0.0

## Prediction

In [8]:
y_pred = model.predict(X_test)
df_ans = pd.DataFrame({"ImageId":list(range(1,28001)), "Label": np.argmax(y_pred, axis=1)})
df_ans.to_csv('ans_nn.csv',index=False)

875/875 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step


Kaggle score is 0.99171.